## Handling my hours

The following notebook handles my hours exported as csv from toggl. This presumes that you have a notebook running with pandas installed (developed on windows with anaconda install) and the exported hours as csv in the same directory. The raw data will not be commited to the repo.

In [1]:
import pandas as pd
hours = pd.read_csv('Toggl_time_entries_2018-01-01_to_2018-12-31.csv')

In [2]:
hours.head()

,User,Email,Client,Project,Task,Description,Billable,Start date,Start time,End date,End time,Duration,Tags,Amount ()
0,Anders Hurum,andershurum@gmail.com,Sensario,Work,NaN,"meeting, modem, telitsim, contiki",No,2018-01-04,08:00:53,2018-01-04,15:41:53,07:41:00,mobile,NaN
1,Anders Hurum,andershurum@gmail.com,Sensario,Work,NaN,"modem, contiki",No,2018-01-05,07:20:30,2018-01-05,15:00:30,07:40:00,mobile,NaN
2,Anders Hurum,andershurum@gmail.com,Sensario,Work,NaN,"modem, contiki",No,2018-01-08,07:15:09,2018-01-08,15:35:09,08:20:00,mobile,NaN
3,Anders Hurum,andershurum@gmail.com,Sensario,Work,NaN,"modem issue#32, contiki docs",No,2018-01-09,07:20:07,2018-01-09,15:30:07,08:10:00,mobile,NaN
4,Anders Hurum,andershurum@gmail.com,Sensario,Work,NaN,"modem#32, contiki UML",No,2018-01-10,06:30:35,2018-01-10,14:40:35,08:10:00,mobile,NaN


In [3]:
hours.index = pd.to_datetime(hours['Start date'], format='%Y-%m-%d')
hours['Duration'] = hours['Duration'].apply(pd.to_timedelta)
hours.info() # Check that df is now datatimeindex

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 165 entries, 2018-01-04 to 2018-09-21
Data columns (total 14 columns):
User           165 non-null object
Email          165 non-null object
Client         165 non-null object
Project        165 non-null object
Task           0 non-null float64
Description    165 non-null object
Billable       165 non-null object
Start date     165 non-null object
Start time     165 non-null object
End date       165 non-null object
End time       165 non-null object
Duration       165 non-null timedelta64[ns]
Tags           165 non-null object
Amount ()      0 non-null float64
dtypes: float64(2), object(11), timedelta64[ns](1)
memory usage: 19.3+ KB


In [4]:
# Group and get the interesting columns to a new df
grouped = hours.groupby(hours.index)
df = pd.DataFrame()
df['Description'] = grouped['Description'].apply(list).apply(lambda col: ', '.join(col))
df['Duration'] = grouped['Duration'].sum()
df.head(20)

,Description,Duration
Start date,,
2018-01-04,"meeting, modem, telitsim, contiki",07:41:00
2018-01-05,"modem, contiki",07:40:00
2018-01-08,"modem, contiki",08:20:00
2018-01-09,"modem issue#32, contiki docs",08:10:00
2018-01-10,"modem#32, contiki UML",08:10:00
2018-01-11,"modem, utilities",09:45:00
2018-01-12,"modem, testing",07:30:00
2018-01-14,device testing,00:40:00
2018-01-15,"modem testing, uml",08:10:00


In [5]:
# Reindex the df
idx = pd.date_range(hours.index[0], hours.index[-1])
df = df.reindex(idx, fill_value=0)
df['Description'] = df['Description'].replace(0, 'No description')
df['Duration'] = df['Duration'].dt.total_seconds().apply(lambda col: (col * 1.05) / 3600 )
df.head()

,Description,Duration
2018-01-04,"meeting, modem, telitsim, contiki",8.0675
2018-01-05,"modem, contiki",8.0500
2018-01-06,No description,0.0000
2018-01-07,No description,0.0000
2018-01-08,"modem, contiki",8.7500


In [6]:
# Export the df
df.to_csv('final.csv')